<a href="https://colab.research.google.com/github/TienNguyen93/project-1/blob/main/cost_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hospital Provider Cost Analysis**

## Import library

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from time import time

## Load dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
path = '/content/drive/MyDrive/dataset/hosp_cost_2022.csv'

prov_cost_df = pd.read_csv(path)
prov_cost_df.head()

,rpt_rec_num,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,...,Net Income from Service to Patients,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone CHIP,Stand-Alone CHIP Charges
0,756569,500051,OVERLAKE HOSPITAL MEDICAL CENTER,1035 116TH AVENUE NE,BELLEVUE,WA,98004-,KING,42644.0,U,...,"$-97,615,780","$61,450,738","$-36,165,042",NaN,"$-36,165,042",0.24,"$18,153,864","$175,634,546",NaN,NaN
1,746892,190146,EAST JEFFERSON GENERAL HOSPITAL,4200 HOUMA,METAIRIE,LA,70006,NaN,35380.0,U,...,"$-81,651,873","$12,804,077","$-68,847,796",NaN,"$-68,847,796",0.25,"$21,647,443","$148,931,816",NaN,NaN
2,744941,310111,CENTRASTATE MEDICAL CENTER,901 WEST MAIN STREET,FREEHOLD,NJ,07728,MONMOUTH,35154.0,U,...,"$-71,333,948","$75,791,218","$4,457,270","$318,112","$4,139,158",0.29,"$21,871,493","$130,400,969",NaN,NaN
3,732016,110030,PIEDMONT CARTERSVILLE MEDICAL CENTER,960 JOE FRANK HARRIS PARKWAY,CARTERSVILLE,GA,30120,BARTOW,12060.0,U,...,"$3,943,456","$2,101,116","$6,044,572",NaN,"$6,044,572",0.09,"$10,348,276","$144,493,159","$4,715","$80,595"
4,763385,452032,VIBRA HOSPITAL OF CLEAR LAKE,709 MEDICAL CTR BLVD,WEBSTER,TX,77598,HARRIS,26420.0,U,...,"$-2,433,293","$365,836","$-2,067,457",NaN,"$-2,067,457",NaN,NaN,NaN,NaN,NaN


In [5]:
prov_cost_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Columns: 117 entries, rpt_rec_num to Stand-Alone CHIP Charges
dtypes: float64(4), int64(4), object(109)
memory usage: 5.4+ MB


In [6]:
prov_cost_df.describe()

,rpt_rec_num,Provider CCN,Medicare CBSA Number,Provider Type,Type of Control,DRG Amounts Other Than Outlier Payments,Outlier Payments For Discharges,Cost To Charge Ratio
count,6064.000000,6064.000000,5981.000000,6064.000000,6064.000000,0.0,0.0,4600.000000
mean,754291.104387,269404.638522,53018.188263,1.708773,4.178595,NaN,NaN,1.349850
std,10770.303579,160322.536510,34012.617263,1.431237,3.200321,NaN,NaN,53.870607
min,717629.000000,10001.000000,10180.000000,1.000000,1.000000,NaN,NaN,0.040000
25%,745140.750000,140157.250000,26420.000000,1.000000,2.000000,NaN,NaN,0.200000
50%,755650.500000,260021.000000,39300.000000,1.000000,2.000000,NaN,NaN,0.290000
75%,762052.750000,393035.250000,99916.000000,1.000000,5.000000,NaN,NaN,0.480000
max,776832.000000,713029.000000,99966.000000,10.000000,13.000000,NaN,NaN,3636.890000


### **Dataset description**
* 6063 entries, 117 features

## Data cleaning

### Handling missing data


In [8]:
nan_percentage = (prov_cost_df.isnull().sum() / len(prov_cost_df)) * 100

In [11]:
nan_percentage = nan_percentage.sort_values(ascending=False)

In [12]:
nan_percentage

,0
Outlier Payments For Discharges,100.000000
DRG Amounts Other Than Outlier Payments,100.000000
Hospital Total Days Title V For Adults & Peds,97.955145
Total Discharges Title V,97.922164
Hospital Total Discharges Title V For Adults & Peds,97.922164
...,...
Provider Type,0.000000
Type of Control,0.000000
Fiscal Year Begin Date,0.000000
Provider CCN,0.000000


#### Remove `Outlier Payments For Discharges` and `DRG Amounts Other Than Outlier Payments` due to 100% of NaN

In [19]:
prov_cost_df.drop(['Outlier Payments For Discharges', 'DRG Amounts Other Than Outlier Payments'], axis=1, inplace=True)

In [22]:
prov_cost_df.columns

Index(['rpt_rec_num', 'Provider CCN', 'Hospital Name', 'Street Address',
       'City', 'State Code', 'Zip Code', 'County', 'Medicare CBSA Number',
       'Rural Versus Urban',
       ...
       'Net Income from Service to Patients', 'Total Other Income',
       'Total Income', 'Total Other Expenses', 'Net Income',
       'Cost To Charge Ratio', 'Net Revenue from Medicaid', 'Medicaid Charges',
       'Net Revenue from Stand-Alone CHIP', 'Stand-Alone CHIP Charges'],
      dtype='object', length=115)